In [1]:
import numpy as np
import pandas as pd
from ckiptagger import data_utils, WS, POS, NER, construct_dictionary
import pprint

In [2]:
#Tagger斷詞器路徑設定
ws_driver = WS("./data/", disable_cuda=True)
pos_driver = POS("./data/", disable_cuda=True)
ner_driver = NER("./data/", disable_cuda=True)

c:\ProgramData\Miniconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
c:\ProgramData\Miniconda3\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
c:\ProgramData\Miniconda3\lib\site-packages\ckiptagger\model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name

In [3]:
#建立Tagger用專有詞典，用來調整權重
word_to_weight = {
    '愛盲': 2,
    '工坊': 1.5,
    '庇護工場': 2,
    '庇護': 1.5,
    '工場': 1,
    '白手杖': 2,
    '視能': 2,
    '身障': 1,
    '護康': 1.5,
    '視障': 2,
    '定向': 1.5,
    '輔具': 1.5,
    '身心': 1,
    '障礙': 1,
    '按摩師': 1.5,
    '按摩': 1,
    '厝邊': 1,
    '盲用': 1.5,
    '黃卡': 1
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'愛盲': 2.0, '工坊': 1.5, '庇護': 1.5, '工場': 1.0, '視能': 2.0, '身障': 1.0, '護康': 1.5, '視障': 2.0, '定向': 1.5, '輔具': 1.5, '身心': 1.0, '障礙': 1.0, '按摩': 1.0, '厝邊': 1.0, '盲用': 1.5, '黃卡': 1.0}), (3, {'白手杖': 2.0, '按摩師': 1.5}), (4, {'庇護工場': 2.0})]


In [4]:
#寫一個func去除某些詞性的詞彙，等同篩除停用詞。
#注意這個func的input需要ws&pos處理過的物件
def wordfilter(sentence_ws, sentence_pos):
  short_with_pos = []
  short_sentence = []
  stop_pos = set(['Nep', 'Nh']) # 不保留的詞性包括指代定詞、 代名詞
  for word_ws, word_pos in zip(sentence_ws, sentence_pos):
    # 只留名詞和動詞
    is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
    # 去掉某些詞性
    is_not_stop_pos = word_pos not in stop_pos
    # 去除只有單個字的詞
    # is_not_one_charactor = not (len(word_ws) == 1)
    # 組成串列
    # if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
    if is_N_or_V and is_not_stop_pos:
      short_with_pos.append(f"{word_ws}({word_pos})")
      short_sentence.append(f"{word_ws}")
  return (" ".join(short_sentence), " ".join(short_with_pos))

#把ws_driver製作成function調用比較方便
def ws_func(ct_str:list):
  ws_list = ws_driver(ct_str, 
            sentence_segmentation = True, 
            segment_delimiter_set = {"，", "。", "！", "？", "：", "；", "…", "、"}, 
            recommend_dictionary = dictionary)
  return ws_list

In [5]:
data = pd.read_csv('taiwan_charity_news.csv', encoding='UTF-8', index_col='Index')

In [6]:
data.head(3)

,Date,Title,Media,Content,Related,Topic_test,Stance_label
Index,,,,,,,
1,2017/1/9,視障弟月薪僅800？ 勞工局：是見習生獎金，不是薪資。,ETtoday_News,一名雙眼失明的闕小弟因為父親入獄、肢體障礙的母親也遠在高雄，只剩他和弟弟2人在北部生活，只好...,1,0,neutral
2,2017/1/13,開發金，邀九企業挺公益。,UDN_Database,開發金控多年來透過旗下中華開發工銀文教基金會及凱基社會慈善基金會，辦理各項公益活動。「年度公...,1,0,neutral
3,2017/3/3,月亮的孩子加強視力，把握黃金期。,UDN_Database,白化症患者體內缺乏黑色素，頭髮和皮膚雪白，因眼睛畏光、在夜間活動相對較舒適，印地安人稱他們為...,1,0,neutral


In [7]:
#清理內文中的一些髒東西
for txts in data['Content']:
    data['Content'] = data['Content'].replace(' ', '').replace(',', '，').replace('(', '（').replace(')', '）').replace('……', '').replace('\n', '')
#
for txts in data['Title']:
    data['Title'] = data['Title'].replace(' ', '').replace(',', '，').replace('(', '（').replace(')', '）').replace('……', '')

In [36]:
#取新聞稿出來檢查
# test_string =data['Content'].loc[data.index == 97].values
# print(test_string)

['「天下的媽媽都是一樣的」，誰說視障女性無法勝任媽媽的角色？\n愛盲基金會表示，基金會成立近30年，看見無數視障家庭面臨的困難與需求，也見證他們的努力，不少許多視障女性也面臨職場與家庭兩頭燒的諸多困境，愛盲服務的其中一位視障媽媽林佳臻，在視障、單親、低收入戶等窘迫條件下，15年來，不但一手撐起整個家、帶大4個孩子，甚至考取多項證照，實現自身志趣。\n愛盲基金會指出，在台灣領有視障手冊的近6萬人中，約87%的人屬於中途失明，林佳臻在15年前因生產過程併發子癇前症，造成視神經萎縮，雙眼視力值只剩0.3，視野中間有黑影擋住，當時身無分文，連辦理低收證明所需的工本費都湊不足。林佳臻說，當時全身上下連5塊錢都沒有，心想如果找到工作，一定要好好做。\n為了照顧一對龍鳳胎及另兩名幼女，林佳臻用米湯混奶粉以節省開銷，並爭取成為公園清潔代賑工，剛開始打掃時，常跌得滿身傷，總掃不乾淨，曾氣餒到想要摔掉掃把，但為了孩子咬牙忍下，一支掃把拿了12年，終於熬到龍鳳胎兒女今年升高一，放學後會主動到公園幫媽媽一起打掃。\n林佳臻說，眼盲並不可怕，心盲才是最可悲的，只要自己走出來，尋求可能的幫助，這也不可恥，她來到愛盲，學會使用白手杖，出門不再容易跌倒；還有可以放大500倍的放大鏡和擴視機，協助她讀書、考上證照。\n愛盲基金會副董事長黃克綸表示，愛盲今年集結了七位視障媽媽的故事，希望透過真實故事，讓大家看見任何一位必須兼顧職場與家庭的母親，都是不容易的，更何況是一位視障母親？很榮幸林佳臻站出來分享親身經歷，讓大家看見視障母親令人超乎想像的強大能量！也呼籲大家能與愛盲一起支持她們，成為所有視障家庭的強力後盾。\n愛盲基金會說，今年受疫情重創，募款陡降近3成，但視障服務的腳步卻不能一日稍停，呼籲社會各界善心人士用行動支持愛盲，一起成為全台灣逾6萬名視障者、逾百萬名低視能者的後援。']


In [37]:
#建立一個測試寫法用的DF
# test = data.iloc[96:98, 0:6]
# test.head(10)

,Date,Title,Media,Content,Related,Stance_label
Index,,,,,,
97,2020/10/22,視力0.3的「超能力」。她靠1把掃把，養活4子女。,United_Daily_News,「天下的媽媽都是一樣的」，誰說視障女性無法勝任媽媽的角色？\n愛盲基金會表示，基金會成立近3...,1,proactive
98,2020/10/22,視力僅剩0.3，愛盲助其考證照、工作、帶大4子長大。,Commercial_Times,視障媽媽林佳臻，在視障、單親、低收入戶等窘迫條件下，15年來，不但一手撐起整個家、帶大四個孩...,1,proactive


In [17]:
#測試單則斷詞
test_line = data['Content'][282]
test_seg = ws_func([test_line])
print(test_seg)

[['為了', '讓', '身心', '障礙', '就業', '者', '更', '了解', '自身', '法令', '權益', '，', '新北市政府', '勞工局', '與', '法扶', '基金會', '合辦', '「', '法令', '交流', '及', '法扶', '制度', '座談會', '」', '，', '邀請', '法扶', '律師', '、', '基金會', '員工', '、', '社工師', '等', '近', '四十', '人', '擔任', '一日', '志工', '，', '走訪', '樸樹', '咖啡', '樹林店', '、', '愛', '盲土城', '工坊', '二', '家', '庇護工場', '，', '勞工局長', '陳瑞嘉', '到場', '鼓勵', '大家', '挽袖', '做', '公益', '。', '\n', '陳瑞嘉', '表示', '，', '新北', '目前', '有', '廿七', '家', '庇護工場', '，', '提供', '超過', '五百', '位', '身障', '者', '工作', '機會', '，', '現在', '年節', '將近', '盼', '消費者', '繼續', '購買', '力挺', '，', '讓', '庇護', '員工', '持續', '穩定', '工作', '有', '經濟', '收入', '，', '此', '次', '感謝', '律師', '志工', '熱情', '參與', '，', '透過', '走進', '庇護工場', '，', '以', '實際', '的', '專業', '能量', '與', '資源', '來', '支持', '身心', '障礙', '者', '。', '法扶', '基金會', '新北分', '會長', '徐偉群', '表示', '，', '明年', '分會', '將', '邁入', '第17', '年', '，', '「', '能夠', '再', '多', '為', '弱勢', '民眾', '多', '做', '些', '什麼', '？', '」', '一直', '是', '持續', '努力', '奮鬥', '的', '核心', '，', '感謝', '勞工局', '共同', '創造', '針對', '弱勢', '民眾', '縝密', '且', '

In [66]:
#寫一個迴圈把content全部丟去給處理，然後新增一欄位放處理好的string
seg_list = []
for ct in data['Content']:
    ws = ws_func([ct])
    pos = pos_driver(ws)
    ner = ner_driver(ws, pos)
    for sentence, sentence_ws, sentence_pos, sentence_ner in zip([ct], ws, pos, ner):
        (seg_str, seg_pos) = wordfilter(sentence_ws, sentence_pos)
        seg_list.append([seg_str])

print(len(seg_list))
pprint.pprint(seg_list)

445
[['一 名 雙眼 失明 闕 小弟 父親 入獄 肢體 障礙 母親 遠 高雄 剩 弟弟 2 人 北部 生活 揹 一 大 籃 糖果 街 上 兜售 經過 好心 '
  '網友 網路 上 幫忙 宣傳 吸引 民眾 愛心 湧入 網傳 許多 資訊 過舊 新北市 勞工局 主動 說明 闕 小弟 現況 勞工局 表示 政府 單位 '
  '101年 10月 受理 小偉 就業 輔導 職業 重建 管理員 評估 後 認為 闕 小弟 能力 適合 職場 有意 推介 愛盲 庇護工場 協助 建立 工作 '
  '態度 技能 表態 願意 102年 2月 職管員 安排 視障者 有聲書 生活 訓練 協助 適應 未來 職場 103年 安排 擔任 愛盲 庇護工場 見習生 '
  '段 期間 開始 利用 下班 後 販賣 口香糖 103年 10月 自願 離開 愛盲 庇護工場 外界 闕 小弟 當時 庇護工場 工作 月薪 800 元 '
  '疑慮 勞工局 表示 庇護工場 薪資 係 依據 身心 障礙 者 權益 保障 法 第40 條 規定 實際 產能 核給 薪資 土城 愛盲 庇護工場 目前 每 '
  '月 平均 薪資 4200 元 功能 好 員工 達 7000 8000 元 該 單位 每 年 調薪 200 300 元 說 月薪 800 元 當時 擔任 '
  '見習生 獎勵金 目前 庇護工場 薪資 外界 不宜 薪資 高低 看 身障 者 庇護 勞動 考量 安置 實現 社會 意義 闕 小弟 每 月 領有 4700 '
  '元 身障 生活 補助 勞工局 社會局 104年 新聞 媒體 披露 後 介入 關心 當時 南勢角 販售 糖果 口香糖 收入 不錯 無 意願 轉業 未來 '
  '繼續 連絡 給予 關懷 有 轉業 想法 勞工局 提供 相關 就業 資源'],
 ['開發 金控 多 年 來 旗下 中華 開發 工銀 文教 基金會 凱基 社會 基金會 辦理 各 項 公益 活動 年度 公益日 每 年 年末 一 大 盛會 '
  '邀請 多 家 社會 企業 公益 廠商 參與 鼓勵 員工 具體 行動 關懷 弱勢 開發金 董事長 張家祝 開發 工銀 文教 基金會 祕書長 吳春台 重視 '
  '鼓勵 員工 積極 參與 具體 實踐 開發 金控 企業 社會 責任 今年 公益日 擴大 活動 規模 邀請 包含 生態 綠 鮮乳坊 台灣 原味

In [67]:
data['Seg_list'] = seg_list
data.head(5)

,Date,Title,Media,Content,Related,Topic_test,Stance_label,Seg_list
Index,,,,,,,,
1,2017/1/9,視障弟月薪僅800？ 勞工局：是見習生獎金，不是薪資。,ETtoday_News,一名雙眼失明的闕小弟因為父親入獄、肢體障礙的母親也遠在高雄，只剩他和弟弟2人在北部生活，只好...,1,0,neutral,[一 名 雙眼 失明 闕 小弟 父親 入獄 肢體 障礙 母親 遠 高雄 剩 弟弟 2 人 北...
2,2017/1/13,開發金，邀九企業挺公益。,UDN_Database,開發金控多年來透過旗下中華開發工銀文教基金會及凱基社會慈善基金會，辦理各項公益活動。「年度公...,1,0,neutral,[開發 金控 多 年 來 旗下 中華 開發 工銀 文教 基金會 凱基 社會 基金會 辦理 各...
3,2017/3/3,月亮的孩子加強視力，把握黃金期。,UDN_Database,白化症患者體內缺乏黑色素，頭髮和皮膚雪白，因眼睛畏光、在夜間活動相對較舒適，印地安人稱他們為...,1,0,neutral,[白化症 患者 體 內 缺乏 黑色素 頭髮 皮膚 雪白 眼睛 畏 光 夜間 活動 相對 舒適...
4,2017/3/17,一銀邀弱勢聽音樂會。,UDN_Database,第一銀行文教基金會日前在台北國際會議中心舉行「經典電影金曲之夜」公益音樂會，超過3千位聽眾讓...,1,0,neutral,[第一 銀行 文教 基金會 日前 台北 國際 會議 中心 舉行 經典 電影 金曲 夜 公益 ...
5,2017/3/24,愛紗婚禮大嘴巴有望合體。薛仕凌：收到口頭邀請。,ETtoday_News,薛仕凌在大嘴巴解散後，加入新東家朝向戲劇發展。他的前團員愛紗預計6月宴客，之前傳出大嘴巴可能...,1,0,proactive,[薛仕凌 大嘴巴 解散 後 加入 新 東家 朝向 戲劇 發展 前 團員 愛紗 預計 6月 宴...


In [68]:
test = seg_list[96:98]
pprint.pprint(test)

[['天下 媽媽 一樣 說 視障 女性 勝任 媽媽 角色 愛盲 基金會 表示 基金會 成立 近 30 年 看見 無數 視障 家庭 面臨 困難 需求 見證 '
  '努力 不少 許多 視障 女性 面臨 職場 家庭 兩 頭 燒 諸多 困境 愛盲 服務 一 位 視障 媽媽 林佳臻 視障 單親 低收入戶 窘迫 條件 下 '
  '15 年 來 撐起 整 個 家 帶 大 4 個 孩子 考取 多 項 證照 實現 志趣 愛盲 基金會 指出 台灣 領有 視障 手冊 6萬 人 中 87% '
  '人 屬於 中途 失明 林佳臻 15 年 前 生產 過程 併發 子癇前 症 造成 視神經 萎縮 雙眼 視力值 剩 0.3 視野 中間 有 黑影 擋住 '
  '當時 身 無 分文 辦理 低收 證明 需 工本費 湊 不足 林佳臻 說 當時 全身 上下 5 塊 錢 沒有 心想 找到 工作 好好 做 照顧 一 對 '
  '龍鳳胎 另 兩 名 幼女 林佳臻 米湯 混 奶粉 節省 開銷 爭取 成為 公園 清潔 代賑工 開始 打掃 時 跌 滿 身 傷 掃 乾淨 氣餒 想要 '
  '摔掉 掃把 孩子 咬牙 忍下 一 支 掃把 拿 12 年 熬到 龍鳳胎 兒女 今年 升高一 放學 後 主動 到 公園 媽媽 打掃 林佳臻 說 眼盲 '
  '可怕 心盲 可悲 走出來 尋求 可能 幫助 可恥 來到 愛盲 學會 使用 白手杖 出門 容易 跌倒 有 放大 500 倍 放大鏡 擴視機 協助 讀書 '
  '考上 證照 愛盲 基金會 副董事長 黃克綸 表示 愛盲 今年 集結 七 位 視障 媽媽 故事 希望 真實 故事 讓 看見 任何 一 位 兼顧 職場 '
  '家庭 母親 容易 一 位 視障 母親 榮幸 林佳臻 站出來 分享 經歷 讓 看見 視障 母親 令 人 超乎 想像 強大 能量 呼籲 愛盲 支持 成為 '
  '所有 視障 家庭 後盾 愛盲 基金會 說 今年 疫情 重創 募款 陡降 3成 視障 服務 腳步 一 日 停 呼籲 社會 各界 善心 人士 行動 支持 '
  '愛盲 成為 全 台灣 逾 6萬 名 視障者 逾 百萬 名 低視能 者 後援'],
 ['視障 媽媽 林佳臻 視障 單親 低收入戶 窘迫 條件 下 15 年 來 撐起 整 個 家 帶 大 四 個 孩子 愛盲 提供 服務 課程 輔具 出門 '
 

In [69]:
data.to_csv('taiwan_charity_news_seglist.csv',encoding='utf-8')

### 增加Title的斷詞

In [15]:
#寫一個迴圈把Title全部丟去給處理，然後新增一欄位放處理好的string
Title_seg = []
for ct in data['Title']:
    ws = ws_func([ct])
    pos = pos_driver(ws)
    ner = ner_driver(ws, pos)
    for sentence, sentence_ws, sentence_pos, sentence_ner in zip([ct], ws, pos, ner):
        (seg_str, seg_pos) = wordfilter(sentence_ws, sentence_pos)
        Title_seg.append([seg_str])

print(len(Title_seg))
pprint.pprint(Title_seg)

445
[['視障 弟 月薪 800 勞工局 見習生 獎金 薪資'],
 ['開發金 邀 九 企業 挺 公益'],
 ['月亮 孩子 加強 視力 把握 黃金期'],
 ['一銀 邀 弱勢 聽 音樂會'],
 ['愛紗 婚禮 大 嘴巴 有望 合體 薛仕凌 收到 邀請'],
 ['原鄉 行腳 社企 搭建 部落 創業路'],
 ['視障 遺傳 醫師 基因 檢測 難 確定'],
 ['執著 王志高 練出 系 微軟 實力'],
 ['臺銀 公益 音樂 饗宴 回饋 貴賓'],
 ['群益 擴大 舉辦 捐血 活動'],
 ['一銀 公益 音樂會 重溫 國片 流金 歲月'],
 ['小嫻 攢 錢 拚子 何守正 健身 防 辣妹'],
 ['新 北 庇護禮 競賽 手工皂 勝出'],
 ['愛盲 手工皂 奪 新北 庇護禮 首獎'],
 ['持續 推 愛盲 任賢齊 黃裕翔 續 10 年 前緣'],
 ['培育 選手 締 佳績 土銀 頒獎 同歡'],
 ['粉筆 灰害 失明 施金順 當 按摩師 摸索 世界'],
 ['粉筆灰 跑到 眼睛 怎麼辦 醫生 學 鹿鼎記 裡 用 菜油'],
 ['徐家 4 姊妹 開設 西瓜 小 學堂 輔導 北埔 地 青年 落地生根'],
 ['一銀 港都 金曲夜 傳遞 地 關懷'],
 ['愷樂 心疼 小嫻 遇到 快樂 愛情'],
 ['國泰證 揪伴 舉辦 義賣'],
 ['保德信 首席 壽險 顧問 獻愛 白手杖 行動'],
 ['商品 有 特色 賣到 趕貨'],
 ['愛心 商品 賣不 動 過年 急'],
 ['新光人壽 愛心 捐血 活動 開跑'],
 ['愛盲 工坊 搬家 空間 大 導覽 體驗 做皂'],
 ['愛 盲工坊 搬到 頂埔站 開放 民眾 互動'],
 ['鬼鬼 體驗 低視能 珍惜 看見 幸福'],
 ['指 劈腿 布魯斯 鬼鬼 7 字 金句 回應'],
 ['鬼鬼 以後 事 難說'],
 ['野村 投信 熱心 公益 力挺 愛 市集'],
 ['黃國輔 大學 很多 乾 妹妹 友人 當 柯 以敏面 爆料 追思 哭 笑'],
 ['新北 庇護工場 公益 產品'],
 ['捐款 回饋禮 涉 左手 賣 右手 愛盲 董座 喊冤 圖利'],
 ['愛盲 儲君 任 要職 謝邦俊 內 舉 避 親'],
 ['張棟樑 拍戲 年底 梁靜茹 探班 聚餐'],
 ['王子 邱

In [20]:
data_seg = pd.read_csv('taiwan_charity_news_seglist.csv', encoding='UTF-8', index_col='Index')

In [21]:
data_seg.head(2)

,Date,Title,Media,Content,Related,Topic_test,Stance_label,Seg_list,News_tags
Index,,,,,,,,,
1,2017/1/9,視障弟月薪僅800？勞工局：是見習生獎金，不是薪資。,ETtoday_News,一名雙眼失明的闕小弟因為父親入獄、肢體障礙的母親也遠在高雄，只剩他和弟弟2人在北部生活，只好...,1,0,neutral,一 名 雙眼 失明 闕 小弟 父親 入獄 肢體 障礙 母親 遠 高雄 剩 弟弟 2 人 北部...,0
2,2017/1/13,開發金，邀九企業挺公益。,UDN_Database,開發金控多年來透過旗下中華開發工銀文教基金會及凱基社會慈善基金會，辦理各項公益活動。「年度公...,1,0,neutral,開發 金控 多 年 來 旗下 中華 開發 工銀 文教 基金會 凱基 社會 基金會 辦理 各 ...,0


In [22]:
data_seg['Title_seg']=Title_seg

In [23]:
data_seg.head(2)

,Date,Title,Media,Content,Related,Topic_test,Stance_label,Seg_list,News_tags,Title_seg
Index,,,,,,,,,,
1,2017/1/9,視障弟月薪僅800？勞工局：是見習生獎金，不是薪資。,ETtoday_News,一名雙眼失明的闕小弟因為父親入獄、肢體障礙的母親也遠在高雄，只剩他和弟弟2人在北部生活，只好...,1,0,neutral,一 名 雙眼 失明 闕 小弟 父親 入獄 肢體 障礙 母親 遠 高雄 剩 弟弟 2 人 北部...,0,[視障 弟 月薪 800 勞工局 見習生 獎金 薪資]
2,2017/1/13,開發金，邀九企業挺公益。,UDN_Database,開發金控多年來透過旗下中華開發工銀文教基金會及凱基社會慈善基金會，辦理各項公益活動。「年度公...,1,0,neutral,開發 金控 多 年 來 旗下 中華 開發 工銀 文教 基金會 凱基 社會 基金會 辦理 各 ...,0,[開發金 邀 九 企業 挺 公益]


In [24]:
data_seg.to_csv('taiwan_charity_news_seglist2.csv', encoding='UTF-8', index=True)